In [168]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser, CommaSeparatedListOutputParser, StrOutputParser
import boto3
import os
import re
import json
from datetime import datetime
from pydantic import BaseModel
import importlib
import shutil
from utils import utils
import pandas as pd
from utils.time_converter import TimeConverter
importlib.reload(utils)
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY")

In [101]:
subject = '수학'
room_id = '6756635bccd21cdc9762967b'

file_keys = utils.get_items('pagecall-text', f'{subject}/{room_id}')
utils.download_items('pagecall-text', file_keys, './downloads')

In [102]:
raw_data = utils.merge_files('./downloads')
shutil.rmtree('./downloads')

In [103]:
teacher_extracted_data = utils.extract_speaker(raw_data, speaker='teacher')
teacher_splited_data = utils.split_sentences(teacher_extracted_data)
student_extracted_data = utils.extract_speaker(raw_data, speaker='student')
student_splited_data = utils.split_sentences(student_extracted_data)

In [104]:
teacher_splited_data = utils.mapping_time(teacher_extracted_data, teacher_splited_data)
teacher_splited_data

[{'idx': 0, 'text': '어 안녕하세요', 'start': '1m 5.9s', 'end': '1m 7.7s'},
 {'idx': 1, 'text': '잘 들려요?', 'start': '1m 5.9s', 'end': '1m 7.7s'},
 {'idx': 2,
  'text': '네 어 좀 수업을 처음 해봐가지고 좀 낮춰져요',
  'start': '1m 9.8s',
  'end': '1m 34.8s'},
 {'idx': 3,
  'text': '일단은 레벨 테스트 틀린 거 먼저 보고 그 다음에 진도를 나갈게요',
  'start': '1m 9.8s',
  'end': '1m 34.8s'},
 {'idx': 4,
  'text': '네 근데 레벨 테스트를 진도를 나갈 수 있을지는 잘 모르겠네요.',
  'start': '1m 9.8s',
  'end': '1m 38.8s'},
 {'idx': 5,
  'text': '빨리 레벨 테스트도 할 게 조금 많아가지고 생각보다는 문제가 어렵더라고요.',
  'start': '1m 39.1s',
  'end': '1m 57.9s'},
 {'idx': 6,
  'text': '근데 연주 자식이 못해서 어려운 건 아니고 객관적으로 봐도 어려운 게 꽤 있는 것 같아요.',
  'start': '1m 58.2s',
  'end': '2m 4.3s'},
 {'idx': 7,
  'text': '저도 한 2개 정도는 푸는데 꽤 오래 걸렸어요',
  'start': '2m 5.8s',
  'end': '2m 34.7s'},
 {'idx': 8,
  'text': '일단은 레벨 테스트 2번 문제 한번 볼게요 상반기 2번 문제요',
  'start': '2m 5.8s',
  'end': '2m 34.7s'},
 {'idx': 9,
  'text': '이게 지금 내심의 정의에 대해서 묻고 있는 거잖아요',
  'start': '2m 5.8s',
  'end': '2m 34.7s'},
 {'idx': 10, 'text': '점 i는

In [105]:
student_splited_data = utils.mapping_time(student_extracted_data, student_splited_data)
student_splited_data

[{'idx': 0, 'text': '음 음 음', 'start': '0m 0.0s', 'end': '0m 4.0s'},
 {'idx': 1, 'text': '안녕하세요', 'start': '0m 37.9s', 'end': '1m 7.3s'},
 {'idx': 2, 'text': '네', 'start': '0m 37.9s', 'end': '1m 7.3s'},
 {'idx': 3, 'text': '괜찮습니다', 'start': '1m 12.4s', 'end': '1m 33.1s'},
 {'idx': 4, 'text': '네', 'start': '1m 12.4s', 'end': '1m 33.1s'},
 {'idx': 5,
  'text': '네 수학 학원을 끊은 지가 2주 정도 돼가지고 그때동안 또 이제 수학 안 하고 있다가 또 해버리니까 문제가 좀 어렵더라고요',
  'start': '1m 42.7s',
  'end': '2m 10.8s'},
 {'idx': 6, 'text': '네?', 'start': '1m 42.7s', 'end': '2m 10.8s'},
 {'idx': 7, 'text': '네.', 'start': '2m 22.6s', 'end': '2m 27.7s'},
 {'idx': 8, 'text': '네.', 'start': '2m 34.6s', 'end': '2m 34.9s'},
 {'idx': 9, 'text': '내 거기 이등분쌈.', 'start': '2m 35.0s', 'end': '2m 39.8s'},
 {'idx': 10, 'text': '맞나 이게?', 'start': '2m 39.9s', 'end': '2m 41.2s'},
 {'idx': 11,
  'text': '네 네 네 말하시다가 끊겨가지고',
  'start': '2m 54.7s',
  'end': '3m 23.4s'},
 {'idx': 12, 'text': '네 네 네 네', 'start': '3m 32.0s', 'end': '3m 51.0s'},
 {'idx': 13, 

In [147]:
teacher_df = pd.DataFrame(teacher_splited_data).rename(columns={"idx": "teacher_idx", "text": "teacher_text"})
student_df = pd.DataFrame(student_splited_data).rename(columns={"idx": "student_idx", "text": "student_text"})

teacher_df['start_ms'] = teacher_df['start'].apply(lambda x: TimeConverter.convert_time_to_milliseconds(x))
student_df['start_ms'] = student_df['start'].apply(lambda x: TimeConverter.convert_time_to_milliseconds(x))

df = pd.concat([teacher_df, student_df], ignore_index=True)
df = df.sort_values(by=["start_ms", "teacher_idx", "student_idx"]).reset_index(drop=True).drop(columns=["start_ms"])
df = df.astype({'teacher_idx': 'Int64', 'student_idx': 'Int64'})
df['time'] = df['start'] + " ~ " + df['end']
df = df.drop(columns=['start', 'end'])
df = df[['teacher_idx', 'student_idx', 'time', 'teacher_text', 'student_text']]

teacher_df = df[df['teacher_text'].notnull()].drop(columns=['student_text', 'student_idx', 'time']).rename(columns={"teacher_idx": "idx", "teacher_text": "text"}).reset_index(drop=True)
student_df = df[df['student_text'].notnull()].drop(columns=['teacher_text', 'teacher_idx', 'time']).rename(columns={"student_idx": "idx", "student_text": "text"}).reset_index(drop=True)
df

,teacher_idx,student_idx,time,teacher_text,student_text
0,<NA>,0,0m 0.0s ~ 0m 4.0s,NaN,음 음 음
1,<NA>,1,0m 37.9s ~ 1m 7.3s,NaN,안녕하세요
2,<NA>,2,0m 37.9s ~ 1m 7.3s,NaN,네
3,0,<NA>,1m 5.9s ~ 1m 7.7s,어 안녕하세요,NaN
4,1,<NA>,1m 5.9s ~ 1m 7.7s,잘 들려요?,NaN
...,...,...,...,...,...
642,475,<NA>,61m 42.3s ~ 62m 11.7s,곱셈 공식 같은 거는 그런 거는 문제도 많이 없으니까 풀고 공식 암기 같은 거를 조...,NaN
643,<NA>,167,61m 59.0s ~ 62m 2.5s,NaN,네 알겠습니다
644,<NA>,168,61m 59.0s ~ 62m 2.5s,NaN,수고하셨습니다
645,476,<NA>,62m 13.0s ~ 62m 18.8s,그러면 다음 시간에 26페이지까지 징고 나갈게요,NaN


In [148]:
teacher_df

,idx,text
0,0,어 안녕하세요
1,1,잘 들려요?
2,2,네 어 좀 수업을 처음 해봐가지고 좀 낮춰져요
3,3,일단은 레벨 테스트 틀린 거 먼저 보고 그 다음에 진도를 나갈게요
4,4,네 근데 레벨 테스트를 진도를 나갈 수 있을지는 잘 모르겠네요.
...,...,...
473,473,그 보면은 개념 설명이 많고 문제는 별로 없거든요
474,474,공식 같은 거는 아직 암기를 못하고 있죠
475,475,곱셈 공식 같은 거는 그런 거는 문제도 많이 없으니까 풀고 공식 암기 같은 거를 조...
476,476,그러면 다음 시간에 26페이지까지 징고 나갈게요


In [119]:
chunks_with_overlap = utils.split_with_overlap(teacher_df, chunk_size=30, overlap=5)

In [120]:
chunks_with_overlap

[[{'idx': 0, 'text': '어 안녕하세요'},
  {'idx': 1, 'text': '잘 들려요?'},
  {'idx': 2, 'text': '네 어 좀 수업을 처음 해봐가지고 좀 낮춰져요'},
  {'idx': 3, 'text': '일단은 레벨 테스트 틀린 거 먼저 보고 그 다음에 진도를 나갈게요'},
  {'idx': 4, 'text': '네 근데 레벨 테스트를 진도를 나갈 수 있을지는 잘 모르겠네요.'},
  {'idx': 5, 'text': '빨리 레벨 테스트도 할 게 조금 많아가지고 생각보다는 문제가 어렵더라고요.'},
  {'idx': 6, 'text': '근데 연주 자식이 못해서 어려운 건 아니고 객관적으로 봐도 어려운 게 꽤 있는 것 같아요.'},
  {'idx': 7, 'text': '저도 한 2개 정도는 푸는데 꽤 오래 걸렸어요'},
  {'idx': 8, 'text': '일단은 레벨 테스트 2번 문제 한번 볼게요 상반기 2번 문제요'},
  {'idx': 9, 'text': '이게 지금 내심의 정의에 대해서 묻고 있는 거잖아요'},
  {'idx': 10, 'text': '점 i는 abc의 내심이다'},
  {'idx': 11, 'text': '그 혹시 내심의 정의가 뭔지 알고 있어요?'},
  {'idx': 12, 'text': '그것도 맞는데 이게 정의가 있고 연준학생?'},
  {'idx': 13, 'text': '혹시 아까도 소리 들렸나요?'},
  {'idx': 14,
   'text': '아 네 저도 갑자기 목통돼가지고 이거 2번 내심이 성질 있고 정의가 있는데 일단 내심의 정의라는 거는 내접원의 중점이잖아요'},
  {'idx': 15,
   'text': '내접원이라는 게 이제 각 이 생각처럼 각 변해적하는 한 오늘 말을 하는 건데 그러면은 정의는 각 변에서의 거리가 같은 점이 맞아요'},
  {'idx': 16, 'text': '그게 정의고 거기서 판이 되는 성질이 말했던 것처럼 내각의 이등분 점인 거죠'},
 

In [121]:
from prompt import question_analyzer

# llm = ChatOpenAI(model='gpt-4o')
llm = ChatOpenAI(
    model='deepseek-chat', 
    openai_api_key=DEEPSEEK_API_KEY, 
    openai_api_base='https://api.deepseek.com',
)

importlib.reload(question_analyzer)
system_prompt = question_analyzer.QuestionAnalyzer(subject, user='선생님').prompt
prompt = ChatPromptTemplate.from_messages([
    ('system', system_prompt),
    ('user', "{user_message}")
])
chain = prompt | llm | StrOutputParser()

# Run it in parallel over chunks
question_analysis_results = chain.batch([
    {"user_message": chunk} for chunk in chunks_with_overlap
])

In [149]:
question_indices = utils.extract_question_indices(question_analysis_results)
question_context = utils.get_question_context(teacher_df.to_dict(orient='records'), question_indices)
question_context

[{'idx': 1,
  'question': '잘 들려요?',
  'context': '어 안녕하세요 잘 들려요? 네 어 좀 수업을 처음 해봐가지고 좀 낮춰져요 일단은 레벨 테스트 틀린 거 먼저 보고 그 다음에 진도를 나갈게요 네 근데 레벨 테스트를 진도를 나갈 수 있을지는 잘 모르겠네요. 빨리 레벨 테스트도 할 게 조금 많아가지고 생각보다는 문제가 어렵더라고요. 근데 연주 자식이 못해서 어려운 건 아니고 객관적으로 봐도 어려운 게 꽤 있는 것 같아요.'},
 {'idx': 11,
  'question': '그 혹시 내심의 정의가 뭔지 알고 있어요?',
  'context': '근데 연주 자식이 못해서 어려운 건 아니고 객관적으로 봐도 어려운 게 꽤 있는 것 같아요. 저도 한 2개 정도는 푸는데 꽤 오래 걸렸어요 일단은 레벨 테스트 2번 문제 한번 볼게요 상반기 2번 문제요 이게 지금 내심의 정의에 대해서 묻고 있는 거잖아요 점 i는 abc의 내심이다 그 혹시 내심의 정의가 뭔지 알고 있어요? 그것도 맞는데 이게 정의가 있고 연준학생? 혹시 아까도 소리 들렸나요? 아 네 저도 갑자기 목통돼가지고 이거 2번 내심이 성질 있고 정의가 있는데 일단 내심의 정의라는 거는 내접원의 중점이잖아요 내접원이라는 게 이제 각 이 생각처럼 각 변해적하는 한 오늘 말을 하는 건데 그러면은 정의는 각 변에서의 거리가 같은 점이 맞아요 그게 정의고 거기서 판이 되는 성질이 말했던 것처럼 내각의 이등분 점인 거죠'},
 {'idx': 13,
  'question': '혹시 아까도 소리 들렸나요?',
  'context': '일단은 레벨 테스트 2번 문제 한번 볼게요 상반기 2번 문제요 이게 지금 내심의 정의에 대해서 묻고 있는 거잖아요 점 i는 abc의 내심이다 그 혹시 내심의 정의가 뭔지 알고 있어요? 그것도 맞는데 이게 정의가 있고 연준학생? 혹시 아까도 소리 들렸나요? 아 네 저도 갑자기 목통돼가지고 이거 2번 내심이 성질 있고 정의가 있는데 일단 내심의 정의라는 거는 내접원의

In [124]:
from prompt import question_classifier

# llm = ChatOpenAI(model='gpt-4o')
llm = ChatOpenAI(
    model='deepseek-chat', 
    openai_api_key=DEEPSEEK_API_KEY, 
    openai_api_base='https://api.deepseek.com',
)

importlib.reload(question_classifier)
system_prompt = question_classifier.QuestionClassifier(subject, user='선생님').prompt
prompt = ChatPromptTemplate.from_messages([
    ('system', system_prompt),
    ('user', "{user_message}")
])
chain = prompt | llm | StrOutputParser()

question_classifier_results = chain.batch([
    {"user_message": chunk} for chunk in question_context
])

In [150]:
for (res,item) in zip(question_classifier_results, question_context):
    item['result'] = res
question_context

[{'idx': 1,
  'question': '잘 들려요?',
  'context': '어 안녕하세요 잘 들려요? 네 어 좀 수업을 처음 해봐가지고 좀 낮춰져요 일단은 레벨 테스트 틀린 거 먼저 보고 그 다음에 진도를 나갈게요 네 근데 레벨 테스트를 진도를 나갈 수 있을지는 잘 모르겠네요. 빨리 레벨 테스트도 할 게 조금 많아가지고 생각보다는 문제가 어렵더라고요. 근데 연주 자식이 못해서 어려운 건 아니고 객관적으로 봐도 어려운 게 꽤 있는 것 같아요.',
  'result': 'False'},
 {'idx': 11,
  'question': '그 혹시 내심의 정의가 뭔지 알고 있어요?',
  'context': '근데 연주 자식이 못해서 어려운 건 아니고 객관적으로 봐도 어려운 게 꽤 있는 것 같아요. 저도 한 2개 정도는 푸는데 꽤 오래 걸렸어요 일단은 레벨 테스트 2번 문제 한번 볼게요 상반기 2번 문제요 이게 지금 내심의 정의에 대해서 묻고 있는 거잖아요 점 i는 abc의 내심이다 그 혹시 내심의 정의가 뭔지 알고 있어요? 그것도 맞는데 이게 정의가 있고 연준학생? 혹시 아까도 소리 들렸나요? 아 네 저도 갑자기 목통돼가지고 이거 2번 내심이 성질 있고 정의가 있는데 일단 내심의 정의라는 거는 내접원의 중점이잖아요 내접원이라는 게 이제 각 이 생각처럼 각 변해적하는 한 오늘 말을 하는 건데 그러면은 정의는 각 변에서의 거리가 같은 점이 맞아요 그게 정의고 거기서 판이 되는 성질이 말했던 것처럼 내각의 이등분 점인 거죠',
  'result': 'True'},
 {'idx': 13,
  'question': '혹시 아까도 소리 들렸나요?',
  'context': '일단은 레벨 테스트 2번 문제 한번 볼게요 상반기 2번 문제요 이게 지금 내심의 정의에 대해서 묻고 있는 거잖아요 점 i는 abc의 내심이다 그 혹시 내심의 정의가 뭔지 알고 있어요? 그것도 맞는데 이게 정의가 있고 연준학생? 혹시 아까도 소리 들렸나요? 아 네 저도 갑자기 목통돼가지고 이거

In [151]:
df

,teacher_idx,student_idx,time,teacher_text,student_text
0,<NA>,0,0m 0.0s ~ 0m 4.0s,NaN,음 음 음
1,<NA>,1,0m 37.9s ~ 1m 7.3s,NaN,안녕하세요
2,<NA>,2,0m 37.9s ~ 1m 7.3s,NaN,네
3,0,<NA>,1m 5.9s ~ 1m 7.7s,어 안녕하세요,NaN
4,1,<NA>,1m 5.9s ~ 1m 7.7s,잘 들려요?,NaN
...,...,...,...,...,...
642,475,<NA>,61m 42.3s ~ 62m 11.7s,곱셈 공식 같은 거는 그런 거는 문제도 많이 없으니까 풀고 공식 암기 같은 거를 조...,NaN
643,<NA>,167,61m 59.0s ~ 62m 2.5s,NaN,네 알겠습니다
644,<NA>,168,61m 59.0s ~ 62m 2.5s,NaN,수고하셨습니다
645,476,<NA>,62m 13.0s ~ 62m 18.8s,그러면 다음 시간에 26페이지까지 징고 나갈게요,NaN


In [164]:
df.to_dict(orient='records')[31:42]

[{'teacher_idx': None,
  'student_idx': 13,
  'time': '4m 6.2s ~ 4m 34.0s',
  'teacher_text': nan,
  'student_text': '네 네 네'},
 {'teacher_idx': 18,
  'student_idx': None,
  'time': '4m 9.2s ~ 4m 41.2s',
  'teacher_text': '그러면은 지금 이번 문제에서 연준학생이 해본거 보면 해외 한거를 보면은 지금 보면은 이 선본 ai를 이렇게 그어놓고 지금 필기 공유가 되나요',
  'student_text': nan},
 {'teacher_idx': 19,
  'student_idx': None,
  'time': '4m 14.8s ~ 4m 41.2s',
  'teacher_text': '여기 보면은 여기 40도 있고 그 다음에 여기 25도 이렇게 있잖아요',
  'student_text': nan},
 {'teacher_idx': 20,
  'student_idx': None,
  'time': '4m 14.8s ~ 5m 14.9s',
  'teacher_text': '이렇게 되는게 아니고 이게 이렇게 뇌접원 이렇게 그어지고 그러면은 지금 제가 갖고 있는 여기 선분 두 개가 같으니까 요거 두 개 길이 같단 말이에요',
  'student_text': nan},
 {'teacher_idx': 21,
  'student_idx': None,
  'time': '4m 47.0s ~ 5m 14.9s',
  'teacher_text': '그러면은 여기를 X라고 하고 여기를 Y라고 하면은 삼각형 AIX랑 AIX랑 삼각형 AIY가 합동인 거예요',
  'student_text': nan},
 {'teacher_idx': 22,
  'student_idx': None,
  'time': '4m 47.0s ~ 5m 14.9s',
  'teacher_text': '이해가 되나요?',
  'student_text': n

In [171]:
feed_idx = [item['idx'] for item in question_context if item['result'] == 'True']
feed_idx = df[df['teacher_idx'].isin(feed_idx)].index.tolist()

final_feed = utils.get_question_context_v2(df.to_dict(orient='records'), feed_idx)
final_feed

[{'idx': 18,
  'question': '그 혹시 내심의 정의가 뭔지 알고 있어요?',
  'context': [{'time': '1m 58.2s ~ 2m 4.3s',
    'teacher_text': '근데 연주 자식이 못해서 어려운 건 아니고 객관적으로 봐도 어려운 게 꽤 있는 것 같아요.'},
   {'time': '2m 5.8s ~ 2m 34.7s', 'teacher_text': '저도 한 2개 정도는 푸는데 꽤 오래 걸렸어요'},
   {'time': '2m 5.8s ~ 2m 34.7s',
    'teacher_text': '일단은 레벨 테스트 2번 문제 한번 볼게요 상반기 2번 문제요'},
   {'time': '2m 5.8s ~ 2m 34.7s',
    'teacher_text': '이게 지금 내심의 정의에 대해서 묻고 있는 거잖아요'},
   {'time': '2m 5.8s ~ 2m 34.7s', 'teacher_text': '점 i는 abc의 내심이다'},
   {'time': '2m 5.8s ~ 2m 34.7s', 'teacher_text': '그 혹시 내심의 정의가 뭔지 알고 있어요?'},
   {'time': '2m 22.6s ~ 2m 27.7s', 'student_text': '네.'},
   {'time': '2m 34.6s ~ 2m 34.9s', 'student_text': '네.'},
   {'time': '2m 35.0s ~ 2m 39.8s', 'student_text': '내 거기 이등분쌈.'},
   {'time': '2m 38.3s ~ 3m 19.5s', 'teacher_text': '그것도 맞는데 이게 정의가 있고 연준학생?'},
   {'time': '2m 39.9s ~ 2m 41.2s', 'student_text': '맞나 이게?'}]},
 {'idx': 30,
  'question': '무슨 말인지 이해가 좀 되나요?',
  'context': [{'time': '3m 20.2s ~ 3m 25.6s',


In [127]:
final_feed = [{'idx': item['idx'], 'question': item['question'], 'context': item['context']} for item in question_context if item['result'] == 'True']
final_feed

[{'idx': 11,
  'question': '그 혹시 내심의 정의가 뭔지 알고 있어요?',
  'context': '근데 연주 자식이 못해서 어려운 건 아니고 객관적으로 봐도 어려운 게 꽤 있는 것 같아요. 저도 한 2개 정도는 푸는데 꽤 오래 걸렸어요 일단은 레벨 테스트 2번 문제 한번 볼게요 상반기 2번 문제요 이게 지금 내심의 정의에 대해서 묻고 있는 거잖아요 점 i는 abc의 내심이다 그 혹시 내심의 정의가 뭔지 알고 있어요? 그것도 맞는데 이게 정의가 있고 연준학생? 혹시 아까도 소리 들렸나요? 아 네 저도 갑자기 목통돼가지고 이거 2번 내심이 성질 있고 정의가 있는데 일단 내심의 정의라는 거는 내접원의 중점이잖아요 내접원이라는 게 이제 각 이 생각처럼 각 변해적하는 한 오늘 말을 하는 건데 그러면은 정의는 각 변에서의 거리가 같은 점이 맞아요 그게 정의고 거기서 판이 되는 성질이 말했던 것처럼 내각의 이등분 점인 거죠'},
 {'idx': 17,
  'question': '무슨 말인지 이해가 좀 되나요?',
  'context': '그것도 맞는데 이게 정의가 있고 연준학생? 혹시 아까도 소리 들렸나요? 아 네 저도 갑자기 목통돼가지고 이거 2번 내심이 성질 있고 정의가 있는데 일단 내심의 정의라는 거는 내접원의 중점이잖아요 내접원이라는 게 이제 각 이 생각처럼 각 변해적하는 한 오늘 말을 하는 건데 그러면은 정의는 각 변에서의 거리가 같은 점이 맞아요 그게 정의고 거기서 판이 되는 성질이 말했던 것처럼 내각의 이등분 점인 거죠 무슨 말인지 이해가 좀 되나요? 그러면은 지금 이번 문제에서 연준학생이 해본거 보면 해외 한거를 보면은 지금 보면은 이 선본 ai를 이렇게 그어놓고 지금 필기 공유가 되나요 여기 보면은 여기 40도 있고 그 다음에 여기 25도 이렇게 있잖아요 이렇게 되는게 아니고 이게 이렇게 뇌접원 이렇게 그어지고 그러면은 지금 제가 갖고 있는 여기 선분 두 개가 같으니까 요거 두 개 길이 같단 말이에요 그러면은 여기를 X라고 하고 여기

In [172]:
from prompt import digging_v2

llm = ChatOpenAI(
    model='deepseek-chat', 
    openai_api_key=DEEPSEEK_API_KEY, 
    openai_api_base='https://api.deepseek.com',
)

importlib.reload(digging_v2)
system_prompt = digging_v2.Digging(subject).prompt
prompt = ChatPromptTemplate.from_messages([
    ('system', system_prompt),
    ('user', "{user_message}")
])
chain = prompt | llm | StrOutputParser()

# Run it in parallel over chunks
results = chain.batch([
    {"user_message": chunk} for chunk in final_feed
])

In [173]:
results = [json.loads(item.strip('```json').strip('```').strip()) for item in results]

for (res, item) in zip(results, final_feed):
    item['result'] = res['result']
    # item['answer'] = res['answer']
    item['reason'] = res['reason']
final_feed

[{'idx': 18,
  'question': '그 혹시 내심의 정의가 뭔지 알고 있어요?',
  'context': [{'time': '1m 58.2s ~ 2m 4.3s',
    'teacher_text': '근데 연주 자식이 못해서 어려운 건 아니고 객관적으로 봐도 어려운 게 꽤 있는 것 같아요.'},
   {'time': '2m 5.8s ~ 2m 34.7s', 'teacher_text': '저도 한 2개 정도는 푸는데 꽤 오래 걸렸어요'},
   {'time': '2m 5.8s ~ 2m 34.7s',
    'teacher_text': '일단은 레벨 테스트 2번 문제 한번 볼게요 상반기 2번 문제요'},
   {'time': '2m 5.8s ~ 2m 34.7s',
    'teacher_text': '이게 지금 내심의 정의에 대해서 묻고 있는 거잖아요'},
   {'time': '2m 5.8s ~ 2m 34.7s', 'teacher_text': '점 i는 abc의 내심이다'},
   {'time': '2m 5.8s ~ 2m 34.7s', 'teacher_text': '그 혹시 내심의 정의가 뭔지 알고 있어요?'},
   {'time': '2m 22.6s ~ 2m 27.7s', 'student_text': '네.'},
   {'time': '2m 34.6s ~ 2m 34.9s', 'student_text': '네.'},
   {'time': '2m 35.0s ~ 2m 39.8s', 'student_text': '내 거기 이등분쌈.'},
   {'time': '2m 38.3s ~ 3m 19.5s', 'teacher_text': '그것도 맞는데 이게 정의가 있고 연준학생?'},
   {'time': '2m 39.9s ~ 2m 41.2s', 'student_text': '맞나 이게?'}],
  'result': '이해점검형 질문',
  'reason': "질문은 학생이 내심의 정의를 알고 있는지 확인하기 위한 것으로, '네' 또는 '아니오'로 대답할 수 있는

In [174]:
with open(f"{room_id}.txt", "w", encoding="utf-8-sig") as file:
    json.dump(final_feed, file, ensure_ascii=False, indent=4)